In [1]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [2]:
from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



In [3]:
script_silver_ddl = """
CREATE SCHEMA IF NOT EXISTS silver;

CREATE TABLE IF NOT EXISTS silver.caracteristiques_clean (
  identifiant_de_l_accident VARCHAR(64),
  date_et_heure TIMESTAMP,
  jour SMALLINT,
  mois SMALLINT,
  annee SMALLINT,
  heure_minute TIME,
  date DATE,
  year_georef SMALLINT,
  lumiere VARCHAR(60),
  code_postal VARCHAR(20),
  code_insee VARCHAR(20),
  departement VARCHAR(6),
  commune VARCHAR(100),
  code_commune VARCHAR(20),
  code_officiel_commune VARCHAR(20),
  nom_officiel_commune VARCHAR(255),
  nom_officiel_commune_arrondissement_municipal VARCHAR(255),
  code_officiel_departement VARCHAR(20),
  nom_officiel_departement VARCHAR(255),
  code_officiel_region VARCHAR(20),
  nom_officiel_region VARCHAR(255),
  code_officiel_epci VARCHAR(20),
  nom_officiel_epci VARCHAR(255),
  localisation VARCHAR(20),
  intersection VARCHAR(60),
  conditions_atmospheriques VARCHAR(50),
  collision VARCHAR(255),
  adresse VARCHAR(512),
  gps VARCHAR(100),
  latitude DOUBLE PRECISION,
  longitude DOUBLE PRECISION,
  coordonnees VARCHAR(255),
  numero VARCHAR(64),
  insert_ts TIMESTAMP DEFAULT now(),
  source VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS silver.lieux_clean(
  identifiant_de_l_accident VARCHAR(64),
  categorie_route SMALLINT,
  voie VARCHAR(128),
  v1 VARCHAR(32),
  v2 VARCHAR(32),
  circulation SMALLINT,
  nombre_de_voies SMALLINT,
  voie_reservee SMALLINT,
  profil SMALLINT,
  pr VARCHAR(64),
  pr1 INT,
  plan SMALLINT,
  largeur_terre_plein_central DOUBLE PRECISION,
  largeur_de_la_chaussee DOUBLE PRECISION,
  surface SMALLINT,
  infrastructure SMALLINT,
  situation SMALLINT,
  env1 SMALLINT,
  insert_ts TIMESTAMP DEFAULT now(),
  source VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS silver.vehicules_clean (
  identifiant_de_l_accident VARCHAR(64),
  identifiant_vehicule VARCHAR(64),
  sens SMALLINT,
  categorie_vehicule SMALLINT,
  obstacle_fixe_heurte SMALLINT,
  obstacle_mobile_heurte SMALLINT,
  point_de_choc SMALLINT,
  manoeuvre SMALLINT,
  nombre_d_occupants SMALLINT,
  insert_ts TIMESTAMP DEFAULT now(),
  source VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS silver.usagers_clean (
  identifiant_de_l_accident VARCHAR(64),
  identifiant_vehicule VARCHAR(64),
  place VARCHAR(128),
  categorie_d_usager SMALLINT,
  gravite SMALLINT,
  sexe SMALLINT,
  annee_de_naissance SMALLINT,
  motif_trajet SMALLINT,
  existence_equipement_de_securite VARCHAR(8),
  utilisation_equipement_de_securite SMALLINT,
  localisation_du_pieton SMALLINT,
  action_pieton SMALLINT,
  pieton_seul_ou_non SMALLINT,
  insert_ts TIMESTAMP DEFAULT now(),
  source VARCHAR(255)
);

"""
engine = get_engine(DB_URL) 
execute_script(engine, script_silver_ddl)




INFO:__main__:Exécution du script SQL...
INFO:__main__:Script SQL exécuté avec succès
INFO:__main__:Connexion fermée et engine libéré.


In [4]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)


INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.


(170, 'postgres', 'db_accident', 'idle', 'SET search_path = public,public,"$user"')
(171, 'postgres', 'db_accident', 'idle', 'select c.oid,pg_catalog.pg_total_relation_size(c.oid) as total_rel_size,pg_catalog.pg_relation_size(c.oid) as rel_size\nFROM pg_class c\nWHERE c.relnamespace=$1')
